In [ ]:
# mysql connection
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [1]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("feed_ods") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "1g") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/06/01 14:08:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 14:08:07 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [ ]:
# get all items from db, and create an rdd
from datetime import datetime
date_string = datetime.today().strftime('%Y-%m-%d')
cur = mydb.cursor()
cur.execute(f"SELECT items.*,{date_string} as date from items")
result = cur.fetchall()
all_items = sc.parallelize(result)
print(all_items.count())

In [4]:
from pyspark.sql.types import DateType, TimestampType, StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("feed_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True),
    StructField("rec_algo", StringType(), True),
    StructField("timestamp", DateType(), True),
    StructField("date", StringType(), True) 
])
item_df = all_items.toDF(schema)
# spark.createDataFrame([],schema).write.mode("overwrite").partitionBy("date").saveAsTable("feed_ods")

23/06/01 14:08:45 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [ ]:
item_df.write.mode("overwrite").partitionBy("date").saveAsTable("feed_ids")

In [ ]:
spark.sql("show tables").show()

In [5]:
spark.stop()
mydb.close()

NameError: name 'mydb' is not defined